In [1]:
from snmp_util import SNMP_runner
from data_struct import Field
import subprocess
from jupyter_util import putting_clipboard as clipboard
import random

In [2]:
%load_ext autoreload

In [9]:
%autoreload 2

In [4]:
# test connected
command = "snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 sysUpTime"
token = subprocess.Popen(command)
print('Command success: {}'.format(token.returncode is None))
print(subprocess.check_output(command).decode('utf-8'))

Command success: True
DISMAN-EVENT-MIB::sysUpTimeInstance = Timeticks: (6503612) 18:03:56.12



In [5]:
snmp = SNMP_runner()
snmp.walk("sysUpTime")
snmp.clear_clipboard()

snmp runtime path: C:\usr\bin
============= init =============

C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 sysUpTime
DISMAN-EVENT-MIB::sysUpTimeInstance = Timeticks: (6503791) 18:03:57.91



In [ ]:
snmp.clear_clipboard()
# set table, or multi setting
"""
iswDhcpClassOption.3 = 2  
iswDhcpClassClientId.3 = "11-22-33-AA-BB-CC"  
iswDhcpClassRowStatus.1 = 4
"""
set_fields = [
    Field(fname='iswDhcpClassOption', val=2, dtype='int'),
    Field(fname='iswDhcpClassClientId', val="11-22-33-AA-BB-CC", dtype='str'),  # d 1 enable
    #Field(fname='iswStaticRouteGateway', val=f"172.16.10.{random.randint(1,254)}", dtype='ip'),  # 1 .. 200000000, d 20000
    Field(fname='iswDhcpClassRowStatus', val=4),  #d 0
]

idx = 3
walk_root = "iswDhcpClassTable"

snmp.print2copyboard("# show")
snmp.walk(walk_root)

snmp.print2copyboard(f"# set entry {idx}")
with snmp as history:
    snmp.set_table(idx=idx, set_fields=set_fields, row_status=None)
    clipboard(history.get_set_history())
    clipboard(snmp.export_copyboard_and_clear())

snmp.print2copyboard("# show")
snmp.walk(walk_root)

In [ ]:
snmp.get_history()

In [6]:
snmp.clear_clipboard()
# walk
test_obj_name = 'iswPoeInfo'
#snmp.set(test_obj_name, 28, 1)
with snmp as history:
    snmp.walk(test_obj_name)
    clipboard("\r\n".join(history.get_history()))
    clipboard(snmp.export_copyboard_and_clear())
     ## !!

C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeInfo
IMP-HF528-MIB::iswPoeChipName.0 = STRING: "PD69210/BT"
IMP-HF528-MIB::iswPoeChipFWVer.0 = STRING: "354.1200.13"
IMP-HF528-MIB::iswPoeChipStatus.0 = STRING: "Normal"



exc_type, exc_val, exc_tb: None None None


In [136]:
# enable test, disable test, this is for scaler, idx usually 0.
snmp.clear_clipboard()
test_obj_name = "iswSysTime"
test_idx = 0
# 
FIRST_SETTING = "00:00:00"
SECOND_SETTING = "23:59:59"  # if No second test, set it ```None'''
D_TYPE = 'str'
snmp.print2copyboard("# show origin")
with snmp as history:
    snmp.get(test_obj_name, test_idx)
    snmp.print2copyboard(" ========================")
    # first
    snmp.set(test_obj_name, test_idx, FIRST_SETTING, dtype=D_TYPE)
    snmp.get(test_obj_name, test_idx)
    if SECOND_SETTING is not None:
        # second
        snmp.set(test_obj_name, test_idx, SECOND_SETTING, dtype=D_TYPE)
        snmp.get(test_obj_name, test_idx)
    # ("------------")
    clipboard(snmp.export_copyboard_and_clear())# log
    clipboard(history.get_set_history())  # command

# show origin
# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswSysTime.0
IMP-HF528-MIB::iswSysTime.0 = STRING: "00:03:34"

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswSysTime.0 = "00:00:00"
IMP-HF528-MIB::iswSysTime.0 = STRING: "00:00:00"

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswSysTime.0
IMP-HF528-MIB::iswSysTime.0 = STRING: "00:03:34"

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswSysTime.0 = "23:59:59"
IMP-HF528-MIB::iswSysTime.0 = STRING: "23:59:59"

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswSysTime.0
IMP-HF528-MIB::iswSysTime.0 = STRING: "00:00:00"



exc_type, exc_val, exc_tb: None None None


In [7]:
# range val or Pick set test. WITH show origin.
val_range =  [0, 720] #  1..65535 80
test_pick_val = (val_range[0]+val_range[-1])//2
#
DTYPE = 'int'
#
test_obj_name = 'iswPoeMaxPower'
test_idx = 0
# 
snmp.print2copyboard("# show origin")
snmp.get(test_obj_name, test_idx)
snmp.print2copyboard(" ========================")
with snmp as history:
    snmp.set(test_obj_name, test_idx, val_range[0], dtype=DTYPE)
    snmp.get(test_obj_name, test_idx)
    snmp.set(test_obj_name, test_idx, test_pick_val, dtype=DTYPE)
    snmp.get(test_obj_name, test_idx)
    snmp.set(test_obj_name, test_idx, val_range[-1], dtype=DTYPE)
    snmp.get(test_obj_name, test_idx)
    # ------------------------------
    clipboard(snmp.export_copyboard_and_clear())
    #history.show_set_cmd()
    clipboard(history.get_set_history())

# show origin
# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 720

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0 = 0
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 0

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 720

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0 = 360
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 360

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 720

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0 = 720
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 720

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoeMaxPower.0
IMP-HF528-MIB::iswPoeMaxPower.0 = INTEGER: 720



exc_type, exc_val, exc_tb: None None None


In [ ]:
# back to default
test_idx = 0
test_obj_name = 'iswStpMSTRevLevel'
def_val = 0
#
snmp.set(test_obj_name, test_idx, def_val)

In [8]:
snmp.clear_clipboard()
# val list test, just CONTINUE set.
#val_list = range(0, 66+1) #[0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240]
val_list = [0,1,2,3]
#
DTYPE = 'int'
#
test_idx = 0
test_obj_name = 'iswPoePowerManagementMode'
#
snmp.print2copyboard("# show origin")
snmp.get(test_obj_name, test_idx)
snmp.print2copyboard(" ======================")
with snmp as history:
    snmp.test_get_continue_set(test_obj_name, test_idx, val_list, dtype=DTYPE)
    clipboard(snmp.export_copyboard_and_clear())
    clipboard(history.get_set_history())

# show origin
# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode.0
IMP-HF528-MIB::iswPoePowerManagementMode.0 = INTEGER: powerAllocatedConsump(3)

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode.0
IMP-HF528-MIB::iswPoePowerManagementMode.0 = INTEGER: powerAllocatedConsump(3)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode.0 = 0
IMP-HF528-MIB::iswPoePowerManagementMode.0 = INTEGER: powserClassReserved(0)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode.0 = 1
IMP-HF528-MIB::iswPoePowerManagementMode.0 = INTEGER: powerClassConsump(1)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode.0 = 2
IMP-HF528-MIB::iswPoePowerManagementMode.0 = INTEGER: powerAllocatedReserved(2)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswPoePowerManagementMode

exc_type, exc_val, exc_tb: None None None


In [ ]:
# multiple binding get 
# 取某 idx 之多個欄位
from_mib="""
iswIfxHCInOctetsL
iswIfxHCInOctetsH
"""
idx = 8
snmp.clear_clipboard()
fields_list = snmp.multi_string_2_list(from_mib)
with snmp as history:
    _ = snmp.multiple_get(idx, fields_list)
    clipboard("\r\n".join(history.get_history()))
    clipboard(snmp.export_copyboard_and_clear())

In [ ]:
# set value ip
test_idx = 0
test_obj_name = 'iswFTPServerIp'
def_val = '43.21.1.99'
#
#
snmp.print2copyboard("# get origin")
snmp.get(test_obj_name, test_idx, def_val)
with snmp as history:
    snmp.set(test_obj_name, test_idx, def_val)
    clipboard(history.get_set_history())
    clipboard(snmp.export_copyboard_and_clear())

In [ ]:
# string val setting
obj_name = "iswFTPServerIp"
idx = 0
val = "abcdefghijklmnopqrstuvwxyz012345"

with snmp as history:
    snmp.set(obj_name, idx, val, dtype='str')
    snmp.get(obj_name, idx)
    clipboard(history.get_set_history())
    clipboard(snmp.export_copyboard_and_clear())



In [ ]:
mib_field = {'\
iswAgingTime	OBJECT-TYPE\
		SYNTAX		Integer32 ( 10 .. 600 )\
		UNITS		"second"\
		MAX-ACCESS	read-write'}             

In [ ]:
test_form =\
{
    "API":"SG",
    "form_type": "Scale",
}

In [ ]:
test_fields = [
    {"name": "iswAgingTime", "read_only": False, "value_list": [], default_value=300},
]

In [ ]:
[range(10,600)]

In [ ]:
type(range(10, 600))

In [ ]:
cmd = walk_command + "iswTpFdbTable"

In [ ]:
raw_return = subprocess.check_output(cmd).decode('utf-8')
format_prefix = r"C:\Users\User>"
print(f"{format_prefix}{cmd}")  # first command
print(f"{raw_return}")  # command output

In [ ]:
# 使用情境
# 測試 scaler 
scale('abc').get()
scale('abc').set(299)
scale('abc').get()
"""
# get
C:\Users\User>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 300 second
# set
C:\Users\User>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0 = 299
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 299 second
# get
C:\Users\User>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 299 second
"""

In [ ]:
#  mib_parser
raw_mib_txt="""
--1
	iswStpProtocol	OBJECT-TYPE
		SYNTAX		Integer32 { stp (0), rstp (1), mstp (2) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP protocol specification. Default value is stp (0)."
		DEFVAL		{ 0 }
		::= { iswStp 1 }
--2 
	iswStpPriority	OBJECT-TYPE
		SYNTAX		Integer32 ( 0 | 4096 | 8192 | 12288 | 16384 | 20480 | 24576 | 28672 | 
								32768 | 36864 | 40960 | 45056 | 49152 | 53248 | 57344 | 61440 )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP priority. Default is 32768(0x8000). Step is 4096."
		DEFVAL		{ 32768 }
		::= { iswStp 2 }
--3
	iswStpTimeSinceTopologyChange	OBJECT-TYPE
		SYNTAX		Gauge32  
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP time since topology change."
		::= { iswStp 3 } 
--4 
	iswStpTopChanges	OBJECT-TYPE
		SYNTAX		Gauge32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP top changes."
		::= { iswStp 4 } 
--5 
	iswStpDesignatedRoot	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE (8) )
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP designated root."
		::= { iswStp 5 } 
--6
	iswStpBridgeID	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE (8) )
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP bridge port ID."
		::= { iswStp 6 } 
--7   
	iswStpRootCost	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP root cost."
		::= { iswStp 7 } 
--8
	iswStpRootPort	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP root port. 0 means NA(no root port)."
		::= { iswStp 8 } 
--9   
	iswStpMaxAge	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP max age."
		::= { iswStp 9 } 
--10  
	iswStpHellowTime	OBJECT-TYPE
		SYNTAX		Integer32
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP hellow time."
		::= { iswStp 10 } 
--11    
	iswStpHoldTime	OBJECT-TYPE
		SYNTAX		Integer32   
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP hold time."
		::= { iswStp 11 } 
--12
	iswStpForwardDelay	OBJECT-TYPE
		SYNTAX		Integer32
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP forward delay."
		::= { iswStp 12 } 
--13    
	iswStpBridgeMaxAge	OBJECT-TYPE
		SYNTAX		Integer32 ( 6 .. 40 ) 
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge max age. Default value is 20.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"
		DEFVAL		{ 20 }
		::= { iswStp 13 } 
--14 
	iswStpBridgeHellowTime	OBJECT-TYPE
		SYNTAX		Integer32 ( 1 .. 10 )
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge hellow time. Default value is 2.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"    
		DEFVAL		{ 2 }
		::= { iswStp 14 } 
--15  
	iswStpBridgeForwardDelay	OBJECT-TYPE
		SYNTAX		Integer32 ( 4 .. 30 )
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge forward delay. Default value is 15.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"
		DEFVAL		{ 15 }
		::= { iswStp 15 } 
--16    
	iswStpBridgeModeSet	OBJECT-TYPE
		SYNTAX		Integer32 { disable (0), enable (1) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge mode set. Default value is disable (0).
					If any ring group is enabled (iswRingProtectRingMode is enable), set this field to enable will be rejected."
		DEFVAL		{ 0 }
		::= { iswStp 16 } 
--17    
	iswStpBpduFilter	OBJECT-TYPE
		SYNTAX		Integer32 { deny (1), flooding (2) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bpdu filter. Default value is deny (1)."  
		DEFVAL		{ 1 }
		::= { iswStp 17 } 
--18
	iswStpRegionName	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE ( 0 .. 32 ) )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP Region Name. Default value is emtpy. Max size is 32 chars, preferably in human-readable form."  
		::= { iswStp 18 } 
--19
	iswStpMSTRevLevel	OBJECT-TYPE
		SYNTAX		Integer32 ( 0 .. 65535 )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"MST Revision Level. Default value is 0."  
		DEFVAL		{ 0 }
		::= { iswStp 19 } 
"""

In [ ]:
_ = [__.replace('\t',' ').strip() for __ in raw_mib_txt.split('\n')]

In [ ]:
for txt in _:
    print(txt)

In [ ]:
#  Hex string valid
def hex_to_bit(hex_string):
    hex_string = hex_string.replace(" ", "")
    int_ = int(hex_string, base=16)
    bit_string = ""
    
    q, p = int_, 0
    while not q < 1:
        bit_string += str(q % 2)
        q = q // 2
        
    bit_string = bit_string[::-1]
    if len(bit_string)%4 != 0:
        bit_string = str(bit_string).zfill((len(bit_string)//4+1)*4)
    
    return bit_string

# count sum of 1 bit 
def cnt_up_bits(bit_string):
    assert len(str(bit_string).replace("0", "").replace("1", "")) == 0  # please input bit string.
    cnt = 0
    for b in bit_string:
        if b == '1':
            cnt += 1
    return cnt

In [ ]:
# calc
cnt_up_bits(hex_to_bit("10"))

In [ ]:
cnt_up_bits(hex_to_bit("81 02 04 08 10 20 40 01 "))

In [ ]:
aa //= 2

In [ ]:
hex(int("00000100", 2).)

In [ ]:
snmp.set("iswIngressPolicerMode", idx=1, val=2)

In [139]:
exist_sheets = ["iswVrrpSystem",
"iswDvmrpConf",
"iswPimConf",
"iswDhcpClassTable",
"iswDRouteTblTable",
"iswMcJoinTable",
"iswVlanIntfTable",
"iswDHCPPoolTable",
"iswDHCPPoolClassAddress",
"iswVrrpGroupStateTable",
"iswVrrpVlanIntfTable",
"iswPortMirrorTxRxTable",
"iswIgmpSystem",
"iswBridgePortTable",
"iswBridgePortIsoTable",
"iswVlanTable",
"iswVlanMember",
"iswTpFdbTable",
"iswFdbStaticTable",
"iswAgingTime",
"iswStp",
"iswStpPortTable",
"iswEthernetPortTable",
"iswJumboFrame",
"iswIfStatsTable",
"iswEtherStatsTable",
"iswPortShaperTable",
"iswQueShaperTable",
"iswSchedulingProfileTable",
"iswSchedulingBindTable",
"iswCosQueMapping",
"iswPolicerIngressColor",
"iswPolicerColorMarking",
"iswIngressPolicerTable",
"iswSdp",
"iswGRingProtect",
"iswSntp",
"iswSyslog",
"iswHttp",
"iswUserStatsTable",
"iswAlarmProfileTable",
"iswHistoryAlarmTable",
"iswCurrentAlarmTable",
"iswEventTable",
"iswAlarmClear",
"iswSysTemperature",
"iswSysLedStatus",
"iswSysDateTime",
"iswInventory",
"iswFTPServer",
"iswSysSaveConfig",
"iswFWDownload",
"iswFWUpgradeScheduleTable",
"iswStaticRouteTable",
"iswGateway",
"iswFWInfoTable",
"iswRestartSNMPAgent",
"iswDBSaveRestore",
"system",
"iswSysRestart",
"iswSdpProductModel",
"iswSntpTimeZone",
"iswLayer3IpForward",
"iswDDMconfig",
"iswDDMITable",
"iswCLISettings"]
index_sheets = ["iswVrrpSystem",
"iswDvmrpIfCreateDelete",
"iswDvmrpConfMode",
"iswDvmrpNeighborTable",
"iswPimConfMode",
"iswPimIfConfTable",
"iswPimNeighborTable",
"iswDHCPClassTable",
"iswDRouteTblTable",
"iswMcJoinTable",
"iswVlanIntfTable",
"iswDHCPPoolTable",
"iswDHCPPoolClassAddress",
"iswVrrpGroupStateTable",
"iswVrrpVlanIntfTable",
"iswIgmpSystem",
"iswPortMirrorTxRxTable",
"iswBridgePortTable",
"iswBridgePortIsoTable",
"iswVlanTable",
"iswVlanMember",
"iswTpFdbTable",
"iswFdbStaticTable/iswFdbStaticDeleteCtrl",
"iswAgingTime",
"iswStp",
"iswStpPortTable",
"iswEthernetPortTable",
"iswJumboFrame",
"iswIfStatsTable",
"iswEtherStatsTable",
"iswPortShaperTable",
"iswQueShaperTable",
"iswSchedulingProfileTable",
"iswSchedulingBindTable",
"iswCosQueMapping",
"iswPolicerIngressColor",
"iswPolicerColorMarking",
"iswIngressPolicerTable",
"iswSdp",
"iswGRingProtect",
"iswSntp",
"iswSyslog",
"iswHttp",
"iswUserStatsTable",
"iswAlarmProfileTable",
"iswHistoryAlarmTable",
"iswCurrentAlarmTable",
"iswEventTable",
"iswAlarmClear",
"iswSysTemperature",
"iswSysLedStatus",
"iswSysDateTime",
"iswInventory",
"iswSysSaveConfig",
"iswFTPServer",
"iswFWDownload",
"iswFWUpgradeScheduleTable",
"iswStaticRouteTable",
"iswGateway",
"iswFWInfoTable",
"iswRestartSNMPAgent",
"iswDBSaveRestore",
"iswSysRestart",
"iswSdpProductModel",
"iswSntpTimeZone",
"iswLayer3IpForward",
"iswDDMconfig",
"iswDDMITable",
"iswCLISettings",
"system"]

In [140]:
index_sheets = ["iswVrrpSystem",
"iswDvmrpIfCreateDelete",
"iswDvmrpConfMode",
"iswDvmrpNeighborTable",
"iswPimConfMode",
"iswPimIfConfTable",
"iswPimNeighborTable",
"iswDHCPClassTable",
"iswDRouteTblTable",
"iswMcJoinTable",
"iswVlanIntfTable",
"iswDHCPPoolTable",
"iswDHCPPoolClassAddress",
"iswVrrpGroupStateTable",
"iswVrrpVlanIntfTable",
"iswIgmpSystem",
"iswPortMirrorTxRxTable",
"iswBridgePortTable",
"iswBridgePortIsoTable",
"iswVlanTable",
"iswVlanMember",
"iswTpFdbTable",
"iswFdbStaticTable/iswFdbStaticDeleteCtrl",
"iswAgingTime",
"iswStp",
"iswStpPortTable",
"iswEthernetPortTable",
"iswJumboFrame",
"iswIfStatsTable",
"iswEtherStatsTable",
"iswPortShaperTable",
"iswQueShaperTable",
"iswSchedulingProfileTable",
"iswSchedulingBindTable",
"iswCosQueMapping",
"iswPolicerIngressColor",
"iswPolicerColorMarking",
"iswIngressPolicerTable",
"iswSdp",
"iswGRingProtect",
"iswSntp",
"iswSyslog",
"iswHttp",
"iswUserStatsTable",
"iswAlarmProfileTable",
"iswHistoryAlarmTable",
"iswCurrentAlarmTable",
"iswEventTable",
"iswAlarmClear",
"iswSysTemperature",
"iswSysLedStatus",
"iswSysDateTime",
"iswInventory",
"iswSysSaveConfig",
"iswFTPServer",
"iswFWDownload",
"iswFWUpgradeScheduleTable",
"iswStaticRouteTable",
"iswGateway",
"iswFWInfoTable",
"iswRestartSNMPAgent",
"iswDBSaveRestore",
"iswSysRestart",
"iswSdpProductModel",
"iswSntpTimeZone",
"iswLayer3IpForward",
"iswDDMconfig",
"iswDDMITable",
"iswCLISettings",
"system"]


In [142]:
index_sheets

{'iswAgingTime',
 'iswAlarmClear',
 'iswAlarmProfileTable',
 'iswBridgePortIsoTable',
 'iswBridgePortTable',
 'iswCLISettings',
 'iswCosQueMapping',
 'iswCurrentAlarmTable',
 'iswDBSaveRestore',
 'iswDDMITable',
 'iswDDMconfig',
 'iswDHCPClassTable',
 'iswDHCPPoolClassAddress',
 'iswDHCPPoolTable',
 'iswDRouteTblTable',
 'iswDvmrpConfMode',
 'iswDvmrpIfCreateDelete',
 'iswDvmrpNeighborTable',
 'iswEtherStatsTable',
 'iswEthernetPortTable',
 'iswEventTable',
 'iswFTPServer',
 'iswFWDownload',
 'iswFWInfoTable',
 'iswFWUpgradeScheduleTable',
 'iswFdbStaticTable/iswFdbStaticDeleteCtrl',
 'iswGRingProtect',
 'iswGateway',
 'iswHistoryAlarmTable',
 'iswHttp',
 'iswIfStatsTable',
 'iswIgmpSystem',
 'iswIngressPolicerTable',
 'iswInventory',
 'iswJumboFrame',
 'iswLayer3IpForward',
 'iswMcJoinTable',
 'iswPimConfMode',
 'iswPimIfConfTable',
 'iswPimNeighborTable',
 'iswPolicerColorMarking',
 'iswPolicerIngressColor',
 'iswPortMirrorTxRxTable',
 'iswPortShaperTable',
 'iswQueShaperTable',
 'is

In [144]:
index_sheets = set(index_sheets)
exist_sheets = set(exist_sheets)
u = index_sheets.union(exist_sheets)

In [145]:
L = u-exist_sheets
R = u-index_sheets


In [146]:
print("only index:", L)
print("only exists:", R)

only index: {'iswPimIfConfTable', 'iswFdbStaticTable/iswFdbStaticDeleteCtrl', 'iswPimConfMode', 'iswDvmrpConfMode', 'iswPimNeighborTable', 'iswDHCPClassTable', 'iswDvmrpIfCreateDelete', 'iswDvmrpNeighborTable'}
only exists: {'iswDhcpClassTable', 'iswPimConf', 'iswDvmrpConf', 'iswFdbStaticTable'}


In [7]:
for i in range(1, 34+1):
    print("::= {{ iswPortMacCounter {} }}".format(i))

::= { iswPortMacCounter 1 }
::= { iswPortMacCounter 2 }
::= { iswPortMacCounter 3 }
::= { iswPortMacCounter 4 }
::= { iswPortMacCounter 5 }
::= { iswPortMacCounter 6 }
::= { iswPortMacCounter 7 }
::= { iswPortMacCounter 8 }
::= { iswPortMacCounter 9 }
::= { iswPortMacCounter 10 }
::= { iswPortMacCounter 11 }
::= { iswPortMacCounter 12 }
::= { iswPortMacCounter 13 }
::= { iswPortMacCounter 14 }
::= { iswPortMacCounter 15 }
::= { iswPortMacCounter 16 }
::= { iswPortMacCounter 17 }
::= { iswPortMacCounter 18 }
::= { iswPortMacCounter 19 }
::= { iswPortMacCounter 20 }
::= { iswPortMacCounter 21 }
::= { iswPortMacCounter 22 }
::= { iswPortMacCounter 23 }
::= { iswPortMacCounter 24 }
::= { iswPortMacCounter 25 }
::= { iswPortMacCounter 26 }
::= { iswPortMacCounter 27 }
::= { iswPortMacCounter 28 }
::= { iswPortMacCounter 29 }
::= { iswPortMacCounter 30 }
::= { iswPortMacCounter 31 }
::= { iswPortMacCounter 32 }
::= { iswPortMacCounter 33 }
::= { iswPortMacCounter 34 }
